In [1]:
import sys
!{sys.executable} -m pip install pandas dash-cytoscape

In [2]:
azg_server = 'localhost:7070'

In [3]:
import anzoGraphClient as azg
import pandas as pd

In [4]:
load_from_http_csv_query='''drop silent graph <nyt_covid_counties> 
;;
PREFIX s:    <http://cambridgesemantics.com/ontologies/DataToolkit#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://anzograph.com/default#>

INSERT { GRAPH <nyt_covid_counties> { 
   ?covidStatIRI a :covidCountyStat ;
        :date  ?date ;
        :county  ?county ;
        :state  ?state ;
        :fips  ?fips ;
        :cases  ?cases ;
        :deaths  ?deaths ;
        :confirmedCases ?confirmed_cases  ;
        :confirmedDeaths ?confirmed_deaths;
        :probableCases ?probable_cases;
        :probableDeaths ?probable_deaths .
 }
}
WHERE
{
   SERVICE <http://cambridgesemantics.com/services/DataToolkit>
   {
        ?data a s:HttpSource ;
            s:url "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv" ;
            ?date xsd:date;
            ?county xsd:string;
            ?state xsd:string;
            ?fips xsd:string;
            ?cases xsd:integer;
            ?deaths xsd:integer;
            ?confirmed_cases xsd:integer;
            ?confirmed_deaths xsd:integer;
            ?probable_cases	xsd:integer;
            ?probable_deaths xsd:integer;    
            .
       BIND(IRI("http://anzograph.com/data/covid/county/{{?fips}}/{{?state}}/{{?date}}") as ?covidStatIRI)
         }
}'''

In [5]:
azg.run_query(azg_server, load_from_http_csv_query)

b''

In [6]:
# this query will count the number of counties we loaded 
count_stats_query='''PREFIX : <http://anzograph.com/default#>
select (count(*) as ?cnt) from <nyt_covid_counties> where {?s a :covidCountyStat}'''

In [7]:
count_df = azg.create_dataframe(azg_server,count_stats_query)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [8]:
count_df

,cnt
0,829428


In [9]:
# uncomment these lines to download the us-counties file and see how many records exist to compare what was loaded. 
#!wget https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv
#!wc -l us-counties.csv

In [10]:
# the above number should match the count_df output above

In [11]:
# clean up 
#!rm us-counties.csv

In [12]:
load_from_api_query='''PREFIX s:		<http://cambridgesemantics.com/ontologies/DataToolkit#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://anzograph.com/default#>

INSERT{
    GRAPH <covid>{
        ?covidtrackerURI a :State_Data ;
            :Current_Hospitalized ?Current_Hospitalized ;
            :Date ?Date ;
            :State ?State ;
            :Total_Deaths ?Total_Deaths ;
            :Total_Hospitalized ?Total_Hospitalized ;
            :Total_Negative ?Total_Negative ;
            :Total_Positive ?Total_Positive ;
            .
    }
}
WHERE
{
  SERVICE TOPDOWN <http://cambridgesemantics.com/services/DataToolkit>
    {
      ?data a s:HttpSource ; #
      s:url "https://api.covidtracking.com/v1/states/daily.json" ; 
      s:timeout 5000 ;
      s:refresh 86400 ; # 1 day
      
      # Output variable bindings
      ?Current_Hospitalized ( "hospitalizedCurrently " xsd:int ) ;
      ?Date ( "date" xsd:date "yyyyMMdd") ;
      ?State ( "state" xsd:string ) ;
      ?Total_Deaths ( "death" xsd:int );
      ?Total_Hospitalized ( "hospitalizedCumulative" xsd:int );
      ?Total_Negative ( "negative" xsd:int );
      ?Total_Positive ( "positive" xsd:int );
      .
    }
   BIND(IRI(CONCAT("http://cambridgesemantics.com/covidproject/", ENCODE_FOR_URI(?State), ENCODE_FOR_URI(STR(?Date)))) AS ?covidtrackerURI)
}'''


In [13]:
azg.run_query(azg_server, load_from_api_query)

b''

In [14]:
# this query will count the number of counties we loaded 
count_states_query='''PREFIX : <http://anzograph.com/default#>
select (count(*) as ?cnt) from <covid> where {?s a :State_Data}'''

In [15]:
count_state_data_df = azg.create_dataframe(azg_server,count_states_query)

In [16]:
count_state_data_df

,cnt
0,16138
